In [26]:
# Load train & dev queries, and paper collection
import pandas as pd
import pickle

# Load paper metadata
with open("subtask4b_collection_data.pkl", "rb") as f:
    papers_df = pickle.load(f)

papers_df["text"] = papers_df["title"] + ". " + papers_df["abstract"]

# Load tweet queries
train_df = pd.read_csv("subtask4b_query_tweets_train.tsv", sep="\t", names=["post_id", "tweet_text", "cord_uid"])
dev_df = pd.read_csv("subtask4b_query_tweets_dev.tsv", sep="\t", names=["post_id", "tweet_text", "cord_uid"])

print(f"Loaded: {len(train_df)} train tweets, {len(dev_df)} dev tweets, {len(papers_df)} papers.")

Loaded: 12854 train tweets, 1401 dev tweets, 7718 papers.


In [27]:
# Load SBERT model
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
#model = SentenceTransformer('multi-qa-MiniLM-L6-dot-v1')
#model = SentenceTransformer('allenai/specter')
#model = SentenceTransformer('msmarco-distilbert-base-tas-b')

#model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
model = SentenceTransformer("/Users/mataonbas/AIR-CheckThat!-GroupProject/CheckThat-ScientificClaimSourceRetrieval/fine_tuned_sbert")

In [28]:
import re

# Define cleaning function. This one is a bit agressive cleaning so result got worsen
#def clean_text(text):
#    text = text.lower()
#    text = re.sub(r"http\\S+|www\\S+", "", text)
#    text = re.sub(r"[@#]\\w+", "", text)
#    text = re.sub(r"[^a-z0-9\\s]", "", text)
#    return text.strip()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)         # remove URLs
    text = re.sub(r"[@#]\w+", "", text)                # remove @mentions and #hashtags
    text = re.sub(r"[^\w\s\-\/]", "", text)            # keep alphanum + dash/slash
    return text.strip()

In [29]:
# Apply cleaning
train_df["clean_tweet_text"] = train_df["tweet_text"].apply(clean_text)
dev_df["clean_tweet_text"] = dev_df["tweet_text"].apply(clean_text)

In [30]:
# Encode cleaned queries
train_query_embeddings = model.encode(
    train_df["clean_tweet_text"].tolist(),
    show_progress_bar=True,
    convert_to_tensor=True
)

dev_query_embeddings = model.encode(
    dev_df["clean_tweet_text"].tolist(),
    show_progress_bar=True,
    convert_to_tensor=True
)

Batches: 100%|██████████| 44/44 [00:23<00:00,  1.86it/s]


In [31]:
# Encode document collection
paper_embeddings = model.encode(papers_df["text"].tolist(), show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 242/242 [26:17<00:00,  6.52s/it] 


In [32]:
# Save paper embeddings to avoid recomputing in future runs
import torch
torch.save(paper_embeddings, "paper_embeddings.pt")

# To load later:
#paper_embeddings = torch.load("paper_embeddings.pt")

In [33]:
from torch.nn.functional import cosine_similarity
from torch.nn.functional import normalize

import torch

#default
#def get_topk_predictions_from_embeddings(query_embeddings, top_k=5):
#    results = []
#    for query_emb in query_embeddings:
#        cos_scores = cosine_similarity(query_emb.unsqueeze(0), paper_embeddings).squeeze()
#        top_indices = torch.topk(cos_scores, k=min(top_k, len(paper_embeddings))).indices.tolist()
#        top_cord_uids = papers_df.iloc[top_indices]["cord_uid"].tolist()
#        results.append(top_cord_uids)
#    return results


def get_topk_predictions_batched(query_embeddings, paper_embeddings, papers_df, top_k=5, batch_size=16):
    predictions = []
    paper_norm = normalize(paper_embeddings, p=2, dim=1)

    for start_idx in range(0, len(query_embeddings), batch_size):
        end_idx = min(start_idx + batch_size, len(query_embeddings))
        query_batch = query_embeddings[start_idx:end_idx]
        query_norm = normalize(query_batch, p=2, dim=1)

        similarity_matrix = torch.matmul(query_norm, paper_norm.T)
        _, top_k_indices = torch.topk(similarity_matrix, k=top_k, dim=1)

        batch_predictions = [
            papers_df.iloc[indices.tolist()]["cord_uid"].tolist()
            for indices in top_k_indices
        ]
        predictions.extend(batch_predictions)

    return predictions

In [34]:
# Run predictions default
#train_df["preds"] = get_topk_predictions_from_embeddings(train_query_embeddings)
#dev_df["preds"] = get_topk_predictions_from_embeddings(dev_query_embeddings)

train_df["preds"] = get_topk_predictions_batched(train_query_embeddings, paper_embeddings, papers_df)
dev_df["preds"] = get_topk_predictions_batched(dev_query_embeddings, paper_embeddings, papers_df)

Evaluation

In [35]:
# Evaluate MRR
def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        scores = []
        for _, row in data.iterrows():
            gold = row[col_gold]
            preds = row[col_pred]
            if isinstance(preds, str):
                try:
                    preds = eval(preds)
                except:
                    preds = []
            if gold in preds[:k]:
                rank = preds[:k].index(gold) + 1
                scores.append(1.0 / rank)
            else:
                scores.append(0.0)
        d_performance[k] = sum(scores) / len(scores) if scores else 0.0
    return d_performance

In [36]:
# Print results default
print("Train MRR:", get_performance_mrr(train_df, "cord_uid", "preds"))
print("Dev MRR:", get_performance_mrr(dev_df, "cord_uid", "preds"))

Train MRR: {1: 0.6729422747782792, 5: 0.7570678388050404, 10: 0.7570678388050404}
Dev MRR: {1: 0.5795860099928622, 5: 0.6492267428027606, 10: 0.6492267428027606}
